In [1]:
import os
spark_version = "spark-3.2.2"
os.environ["SPARK_VERSION"] = spark_version

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-20 22:58:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.44MB/s    in 0.2s    

2022-11-20 22:58:35 (5.44 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Gift_Card").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles

url= "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [4]:
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Gift_Card_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [5]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [6]:
df = df.withColumn("review_date", df["review_date"].cast("date"))

In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [8]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [9]:
df = df.dropna()

num_rows = df.count()

print(f"Number of rows = {num_rows:,}")

Number of rows = 149,080


In [10]:
df.distinct().count()

149080

In [11]:
from pyspark.sql.types import StructField, StructType, DateType, IntegerType, StringType

In [12]:
# Make tables: review_id_table; products; customers; vine_table

# CREATE TABLE review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE 
# );

review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])

review_id_df.show()

review_id_df.printSchema()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R27ZP1F1CD0C3Y|   24371595|B004LLIL5A|     346014806| 2015-08-31|
| RJ7RSBCHUDNNE|   42489718|B004LLIKVU|     473048287| 2015-08-31|
|R1HVYBSKLQJI5S|     861463|B00IX1I3G6|     926539283| 2015-08-31|
|R2HAXF0IIYQBIR|   25283295|B00IX1I3G6|     926539283| 2015-08-31|
| RNYLPX611NB7Q|     397970|B005ESMGV4|     379368939| 2015-08-31|
|R3ALA9XXMBEDZR|   18513645|B004KNWWU4|     326384774| 2015-08-31|
|R3R8PHAVJFTPDF|   22484620|B004LLIKVU|     473048287| 2015-08-31|
|R18WWEK8OIXE30|   14765851|BT00CTP2EE|     775486538| 2015-08-31|
|R1EGUNQON2J277|   18751931|B004LLIKVU|     473048287| 2015-08-31|
|R21Z4M4L98CPU2|   15100528|B004W8D102|     595099956| 2015-08-31|
| R6JH7A117FHFA|    3559726|B004LLIKVU|     473048287| 2015-08-31|
|R1XZHS8M1GCGI7|   23413911|B004KNWWU4|     326384774| 2015-08

In [13]:
# Change nullable value

new_schema = StructType([
   StructField("review_id", StringType(), False),
   StructField("customer_id", IntegerType(), True),
   StructField("product_id", StringType(), True),
   StructField("product_parent", IntegerType(), True),
   StructField("review_date", DateType(), True)
   ])

review_id_df = spark.createDataFrame(review_id_df[["review_id", "customer_id", "product_id", "product_parent", "review_date"]].collect(), new_schema)

review_id_df.printSchema()

review_id_df.show()

root
 |-- review_id: string (nullable = false)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R27ZP1F1CD0C3Y|   24371595|B004LLIL5A|     346014806| 2015-08-31|
| RJ7RSBCHUDNNE|   42489718|B004LLIKVU|     473048287| 2015-08-31|
|R1HVYBSKLQJI5S|     861463|B00IX1I3G6|     926539283| 2015-08-31|
|R2HAXF0IIYQBIR|   25283295|B00IX1I3G6|     926539283| 2015-08-31|
| RNYLPX611NB7Q|     397970|B005ESMGV4|     379368939| 2015-08-31|
|R3ALA9XXMBEDZR|   18513645|B004KNWWU4|     326384774| 2015-08-31|
|R3R8PHAVJFTPDF|   22484620|B004LLIKVU|     473048287| 2015-08-31|
|R18WWEK8OIXE30|   14765851|BT00CTP2EE|     775486538| 2015-08-31|
|R1EGUNQON2J277|   18751931|B004LLIKVU|  

In [14]:
# -- This table will contain only unique values

# CREATE TABLE products (
#   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#   product_title TEXT
# );


products_df = df.select(["product_id", "product_title"])

products_df.count() 

149080

In [15]:

products_df = products_df.distinct()

products_df.show()

products_df.printSchema()

products_df.count()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CHQKNUM|Amazon.com Gift C...|
|B00CRQ4ZOM|Amazon Gift Card ...|
|B00H5BMZ32|Amazon eGift Card...|
|BT00CTP84S|Amazon.com Gift C...|
|B004LLILN2|Amazon eGift Card...|
|B00N59437O|Amazon Gift Card ...|
|B005J0444E|Amazon.com Gift C...|
|B00E1QB2K6|Amazon Gift Card ...|
|B004LLJ6IQ|Amazon Gift Card ...|
|B004LLILQ4|Amazon eGift Card...|
|B0091JL2CQ|Amazon.com Gift C...|
|B0062ONNL6|Amazon Gift Card ...|
|BT00CTP0N2|Amazon.com Gift C...|
|B009I1Z11U|Amazon Video Gift...|
|B009LLWBTM|Amazon Video Gift...|
|B00BWDHS04|Amazon Gift Card ...|
|B0091JKZYW|Amazon.com Gift C...|
|B004LLJ6C2|Amazon Gift Card ...|
|B007V6ETDK|Amazon eGift Card...|
|B005Z3D5OU|Amazon.com Gift C...|
+----------+--------------------+
only showing top 20 rows

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



1779

In [16]:
# Change nullable value

new_schema = StructType([
   StructField("product_id", StringType(), False),
   StructField("product_title", StringType(), True)])

products_df = spark.createDataFrame(products_df[["product_id", "product_title"]].collect(), new_schema)

products_df.printSchema()

products_df.show()

root
 |-- product_id: string (nullable = false)
 |-- product_title: string (nullable = true)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CHQKNUM|Amazon.com Gift C...|
|B00CRQ4ZOM|Amazon Gift Card ...|
|B00H5BMZ32|Amazon eGift Card...|
|BT00CTP84S|Amazon.com Gift C...|
|B004LLILN2|Amazon eGift Card...|
|B00N59437O|Amazon Gift Card ...|
|B005J0444E|Amazon.com Gift C...|
|B00E1QB2K6|Amazon Gift Card ...|
|B004LLJ6IQ|Amazon Gift Card ...|
|B004LLILQ4|Amazon eGift Card...|
|B0091JL2CQ|Amazon.com Gift C...|
|B0062ONNL6|Amazon Gift Card ...|
|BT00CTP0N2|Amazon.com Gift C...|
|B009I1Z11U|Amazon Video Gift...|
|B009LLWBTM|Amazon Video Gift...|
|B00BWDHS04|Amazon Gift Card ...|
|B0091JKZYW|Amazon.com Gift C...|
|B004LLJ6C2|Amazon Gift Card ...|
|B007V6ETDK|Amazon eGift Card...|
|B005Z3D5OU|Amazon.com Gift C...|
+----------+--------------------+
only showing top 20 rows



In [17]:
# -- Customer table for first data set

# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );

customers_df = df.groupBy("customer_id").count()

customers_df = customers_df.withColumnRenamed("count", "customer_count")

customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast("int"))

customers_df.show()

customers_df.printSchema()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12819130|             2|
|   50079826|             2|
|    3399669|             1|
|   23344898|             1|
|   42139031|             1|
|   48994143|             1|
|     768171|             1|
|   29907188|             1|
|   28161223|             1|
|     184483|             1|
|     225566|             1|
|   42628722|             1|
|    2903234|             1|
|   11285702|             2|
|   15423461|             1|
|   30717827|             1|
|   35594973|             1|
|   20157057|             1|
|   27747830|             1|
|   11150152|             1|
+-----------+--------------+
only showing top 20 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [18]:
# Change nullable value

new_schema = StructType([
   StructField("customer_id", IntegerType(), False),
   StructField("customer_count", IntegerType(), True)])

customers_df = spark.createDataFrame(customers_df[["customer_id", "customer_count"]].collect(), new_schema)

customers_df.printSchema()

customers_df.show()

root
 |-- customer_id: integer (nullable = false)
 |-- customer_count: integer (nullable = true)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12819130|             2|
|   50079826|             2|
|    3399669|             1|
|   23344898|             1|
|   42139031|             1|
|   48994143|             1|
|     768171|             1|
|   29907188|             1|
|   28161223|             1|
|     184483|             1|
|     225566|             1|
|   42628722|             1|
|    2903234|             1|
|   11285702|             2|
|   15423461|             1|
|   30717827|             1|
|   35594973|             1|
|   20157057|             1|
|   27747830|             1|
|   11150152|             1|
+-----------+--------------+
only showing top 20 rows



In [19]:
# -- vine table

# CREATE TABLE vine_table (
#   review_id TEXT PRIMARY KEY,
#   star_rating INTEGER,
#   helpful_votes INTEGER,
#   total_votes INTEGER,
#   vine TEXT
# );

vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

vine_table_df.show()

vine_table_df.printSchema()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|
| RNYLPX611NB7Q|          5|            0|          0|   N|
|R3ALA9XXMBEDZR|          5|            0|          0|   N|
|R3R8PHAVJFTPDF|          5|            0|          0|   N|
|R18WWEK8OIXE30|          5|            0|          0|   N|
|R1EGUNQON2J277|          1|            0|          0|   N|
|R21Z4M4L98CPU2|          5|            0|          0|   N|
| R6JH7A117FHFA|          5|            0|          0|   N|
|R1XZHS8M1GCGI7|          5|            1|          1|   N|
|R1DAI0N03SKRJN|          5|            1|          1|   N|
|R2F6SKZOEYQRU3|          5|            

In [28]:
# AWS

my_aws_endpoint = 'mypostgresdb.ct46lxgt7vhw.us-east-2.rds.amazonaws.com' 
my_aws_port_number = '5432' 
my_aws_database_name = 'my_data_class_db' 
my_aws_username = '' 
my_aws_password = '' 

In [25]:
# Define the connection string

jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters

config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

mode = 'append' 

In [22]:
# Make tables: review_id_table; products; customers; vine_table
# Write the dataframe to the appropriate table in your PostgreSQL RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [23]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [26]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [27]:
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)